<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [172]:
import pandas as pd
import psycopg2
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [173]:
# параметры подключения
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432 

In [174]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [175]:
query_3_1 = f'''select 
            COUNT(id) "Количество_вакансий_в_базе"
           from public.vacancies
        '''  

In [176]:
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1

,Количество_вакансий_в_базе
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [177]:
query_3_2 = f'''select DISTINCT
            COUNT(name) "Количество_работодателей"
           from public.employers
        ''' 

In [178]:
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2

,Количество_работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [179]:
query_3_3 = f'''select DISTINCT
            COUNT(id) "Количество_регионов"
           from public.areas
        ''' 

In [180]:
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3

,Количество_регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [181]:
query_3_4 = f'''select DISTINCT
            COUNT(id) "Количество_сфер_деятельности"
           from public.industries
        ''' 

In [182]:
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4

,Количество_сфер_деятельности
0,294


***

Выводы:

В базе вакансий находятся около 50 000 вакансий. При этом количество регионов около 1 400. Количество работодателей составляет около 24 000, которые осуществляют около 300 сфер деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [183]:
query_4_1 = f'''select 
           DISTINCT(a.name) "Регион",
           COUNT(v.id) "Количество_вакансий"
           from public.areas a
           join public.vacancies v on a.id = v.area_id
           GROUP BY a.name
           ORDER BY 2 DESC
           LIMIT 5
        ''' 

In [184]:
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1

,Регион,Количество_вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [185]:
query_4_2 = f'''select 
           COUNT(id) "Количество_вакансий"
           from public.vacancies
           WHERE salary_from IS NOT NULL or salary_to IS NOT NULL
        '''  

In [186]:
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2

,Количество_вакансий
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [187]:
query_4_3 = f'''select 
           ROUND(AVG(salary_from)) "Нижняя_граница_ЗП",
           ROUND(AVG(salary_to)) "Верхняя_граница_ЗП" 
           from public.vacancies
           WHERE salary_from IS NOT NULL or salary_to IS NOT NULL
        '''  

In [188]:
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3

,Нижняя_граница_ЗП,Верхняя_граница_ЗП
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [189]:
query_4_4 = f'''select 
           schedule "Тип_рабочего_графика",
           employment "Тип_трудоустройства",
           COUNT(id) "Количество вакансий"
           from public.vacancies
           GROUP BY 1, 2
           ORDER BY 3 DESC
        '''  

In [190]:
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4

,Тип_рабочего_графика,Тип_трудоустройства,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [191]:
query_4_5 = f'''select 
           experience "Опыт_работы",
           COUNT(id)  "Количество_вакансий"
           from public.vacancies
           GROUP BY 1
           ORDER BY 2 ASC
        '''  

In [192]:
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5

,Опыт_работы,Количество_вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Выводы:

Самое большое количество вакансий представлено в Москве. Далее по убыванию идут Санкт-Петербург и Минск.

Только в половине вакансий указаны границы заработной платы.

Средние значения для нижней и верхней границы зарплатной вилки составляют 71 000 и 111 000 соответственно.

Подавляющее большинство вакансий, а именно около 35 000, с требованиями полного рабочего дня и полной занятостью.

Наибольшее количество вакансий с наименьшим опытом. С максимальным опытом наименьшее число вакансий. Седьмую часть вакансий составляют вакансии от работодателей, которые не требуют опыта работы.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [193]:
query_5_1 = f'''select 
           emp.name "Работодатель",
           COUNT(v.id) "Количество_вакансий"
           from public.vacancies v
           join public.employers emp on emp.id = v.employer_id
           GROUP BY 1
           ORDER BY 2 DESC
           LIMIT 5
        '''  

In [194]:
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1

,Работодатель,Количество_вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [195]:
query_5_2 = f'''select 
           a.name "Регион",
           COUNT(emp.id) "Количество_работодателей",
           COUNT(v.id) "Количество_вакансий"
           from public.areas a
           join public.employers emp on emp.area = a.id
           left join public.vacancies v on a.id = v.area_id
           WHERE v.id IS NULL
           GROUP BY 1
           ORDER BY 2 DESC
        '''  

In [196]:
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2

,Регион,Количество_работодателей,Количество_вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
...,...,...,...
181,Миллерово,1,0
182,"Михайловка (Уфимский район, Башкортостан)",1,0
183,Можайск,1,0
184,Морозовск\n,1,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [197]:
query_5_3 = f'''select
           emp.name "Работодатель",
           COUNT(DISTINCT v.area_id) "Количество_регионов"
           from public.employers emp
           join public.vacancies v on emp.id = v.employer_id
           GROUP BY emp.id 
           ORDER BY 2 DESC
        '''  

In [198]:
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3

,Работодатель,Количество_регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [199]:
query_5_4 = f'''select
           COUNT(emp.id) "Количество работодателей"
           from public.employers emp
           left join public.employers_industries i on i.employer_id = emp.id
           WHERE i.industry_id IS NULL
        '''  

In [200]:
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4

,Количество работодателей
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [201]:
query_5_5 = f'''select
           emp.name "Работодатель",
           COUNT(i.industry_id) "Количество_деятельностей"
           from public.employers emp
           left join public.employers_industries i on i.employer_id = emp.id
           GROUP BY emp.name
           HAVING COUNT(i.industry_id) = 4
           ORDER BY emp.name ASC
           OFFSET 2 LIMIT 1
        '''  

In [202]:
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5

,Работодатель,Количество_деятельностей
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [203]:
query_5_6 = f'''select
         COUNT(emp.id) "Количество_работодателей"
         from public.employers emp
         left join public.employers_industries i on i.employer_id = emp.id
         join public.industries ind on ind.id = i.industry_id
         WHERE ind.name = 'Разработка программного обеспечения'
        '''  

In [204]:
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6

,Количество_работодателей
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [205]:
import pandas as pd
import urllib.parse as req

path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
url

'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'

In [206]:
query_5_7 = f'''
        SELECT 
            a.name "Регион", 
            COUNT(v.id) "Количество_вакансий"
        FROM public.employers e
            JOIN public.vacancies v ON e.id=v.employer_id
            JOIN public.areas a ON a.id=v.area_id 
        WHERE e.name LIKE '%Яндекс%' 
        AND (a.name LIKE '%Москва%' OR a.name LIKE '%Санкт-Петербург%'
        OR a.name LIKE '%Новосибирск%' OR a.name LIKE '%Екатеринбург%' OR a.name LIKE '%Казань%'
        OR a.name LIKE '%Нижний Новгород%' OR a.name LIKE '%Красноярск%' OR a.name LIKE '%Челябинск%'
        OR a.name LIKE '%Самара%' OR a.name LIKE '%Уфа%'OR a.name LIKE '%Ростов-на-Дону%'
        OR a.name LIKE '%Омск%' OR a.name LIKE '%Краснодар%' OR a.name LIKE '%Воронеж%' 
        OR a.name LIKE '%Пермь%' OR a.name LIKE '%Волгоград%')                                                                                         
        GROUP BY a.name

        UNION all
                                                                                                    
        SELECT 
            'Total',
            count(*)
        FROM public.vacancies v
        JOIN public.areas a ON a.id=v.area_id
        JOIN public.employers e ON e.id=v.employer_id
        WHERE e.name LIKE '%Яндекс%' 
        AND (a.name LIKE '%Москва%' OR a.name LIKE '%Санкт-Петербург%'
        OR a.name LIKE '%Новосибирск%' OR a.name LIKE '%Екатеринбург%' OR a.name LIKE '%Казань%'
        OR a.name LIKE '%Нижний Новгород%' OR a.name LIKE '%Красноярск%' OR a.name LIKE '%Челябинск%'
        OR a.name LIKE '%Самара%' OR a.name LIKE '%Уфа%'OR a.name LIKE '%Ростов-на-Дону%'
        OR a.name LIKE '%Омск%' OR a.name LIKE '%Краснодар%' OR a.name LIKE '%Воронеж%' 
        OR a.name LIKE '%Пермь%' OR a.name LIKE '%Волгоград%') 
        ORDER BY 2 ASC
        '''

In [207]:
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7

,Регион,Количество_вакансий
0,Волжский (Волгоградская область),13
1,Омск,21
2,Челябинск,22
3,Красноярск,23
4,Волгоград,24
5,Ростов-на-Дону,25
6,Казань,25
7,Пермь,25
8,Уфа,26
9,Самара,26


***

Выводы:

На первом месте по количеству вакансий находится Яндекс, около 1700 вакансий. Далее идут Ростелеком, Тинькофф, СБЕР и Газпром нефть.

Среди представленных вакансий наибольшее количество работодателей, находится в России.

Наибольшее количество регионов - 181, в которых работодатель публикует вакансии, охватывает Яндекс. Ростелеком охватывает 152 региона. Спецремонт публикует вакансии в 116 регионах.

Почти у 8 000 работодателей не указана сфера деятельности.

Компания 2ГИС, находится на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

У почти 3 500 работодателей в качестве сферы деятельности указана "Разработка программного обеспечения".

Общее количество вакансий от компании "Яндекс" в городах-милионников 504 вакансии. Самое большое количество вакансий в городе Москва, затем в Санкт-Петербурге и Екатеринбурге.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [208]:
query_6_1 = f'''select
           COUNT(id) "Количество_вакансий"
           from public.vacancies
           WHERE name ILIKE '%data%' or name ILIKE '%данн%'
        '''  

In [209]:
df_6_1 = pd.read_sql_query(query_6_1, connection)
df_6_1

,Количество_вакансий
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [210]:
query_6_2 = f'''select
           COUNT(id) "Количество_вакансий"
           from public.vacancies
           WHERE (name ILIKE '%data scientist%' 
                or (name LIKE '%ML%' and name NOT ILIKE '%HTML%')
                or name ILIKE '%исследователь данных%'
                or name ILIKE '%machine learning%' 
                or name ILIKE '%машинн%обучен%' 
                or name ILIKE '%data science%') 
                and (name ILIKE '%junior%'
                or experience ILIKE 'Нет опыта'
                or employment ILIKE 'Стажировка')
           ''' 

In [211]:
df_6_2 = pd.read_sql_query(query_6_2, connection)
df_6_2

,Количество_вакансий
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [212]:
query_6_3 = f'''select
           COUNT(id) "Количество_вакансий"
           from public.vacancies
           WHERE (name ILIKE '%data scientist%' 
                or (name LIKE '%ML%' and name NOT ILIKE '%HTML%')
                or name ILIKE '%исследователь данных%'
                or name ILIKE '%machine learning%' 
                or name ILIKE '%машинн%обучен%' 
                or name ILIKE '%data science%')
                and (key_skills ILIKE '%SQL%' or key_skills ILIKE '%postgres%')
           ''' 

In [213]:
df_6_3 = pd.read_sql_query(query_6_3, connection)
df_6_3

,Количество_вакансий
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [214]:
query_6_4 = f'''select
           COUNT(id) "Количество_вакансий"
           from public.vacancies
           WHERE (name ILIKE '%data scientist%' 
                or (name LIKE '%ML%' and name NOT ILIKE '%HTML%')
                or name ILIKE '%исследователь данных%'
                or name ILIKE '%machine learning%' 
                or name ILIKE '%машинн%обучен%' 
                or name ILIKE '%data science%')
                and key_skills ILIKE '%Python%'
           ''' 

In [215]:
df_6_4 = pd.read_sql_query(query_6_4, connection)
df_6_4

,Количество_вакансий
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [216]:
query_6_5 = f'''select
           AVG(length(key_skills) - length(REPLACE(key_skills, CHR(9), '' )) + 1)  "Среднее_количество_навыков"         
           from public.vacancies
           WHERE (name ILIKE '%data scientist%' 
                or (name LIKE '%ML%' and name NOT ILIKE '%HTML%')
                or name ILIKE '%исследователь данных%'
                or name ILIKE '%machine learning%' 
                or name ILIKE '%машинн%обучен%' 
                or name ILIKE '%data science%')
                and key_skills IS NOT NULL
           ''' 

In [217]:
df_6_5 = pd.read_sql_query(query_6_5, connection)
round(df_6_5, 2)

,Среднее_количество_навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [218]:
query_6_6 = f'''select
           experience "Опыт_работы",
           ROUND(AVG(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0) "Средняя_ЗП"        
           from public.vacancies
           WHERE (name ILIKE '%data scientist%' 
                or (name LIKE '%ML%' and name NOT ILIKE '%HTML%')
                or name ILIKE '%исследователь данных%'
                or name ILIKE '%machine learning%' 
                or name ILIKE '%машинн%обучен%' 
                or name ILIKE '%data science%')
                and experience ILIKE 'От 3 до 6 лет'
           GROUP BY experience
           ''' 

In [219]:
df_6_6 = pd.read_sql_query(query_6_6, connection)
df_6_6
# connection.close()

,Опыт_работы,Средняя_ЗП
0,От 3 до 6 лет,243115.0


***

Выводы:
    
Количество вакансий в базе, в которых требуются специалисты по работе с данными около 1800.

Для начинающего дата-сайентиста, т.е. для сотрудника у которого нет опыта работы или необходима работа на период стажировки, в базе имеется около 50 вакансий.

Количество вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres - 201 вакансия.

В базе вакансий около 350 вакансий для специалиста DS, в которых в качестве ключевого навыка указан язык программирования Python.

Для специалистов DS в вакансиях указано около 6 ключевых навыков.

Размер средней заработной платы для специалиста DS, чей опыт работы составляет от 3 до 6 лет, около 240 000.

# Общий вывод по проекту

Из представленной информации можно сделать вывод, что на текущий момент на рынке труда присутствует большое количество вакансий в различных сферах деятельности, большинство из которых требует полный рабочий день и полную занятость. Чтобы успешно найти работу, необходимо иметь опыт работы и навыки работы с данными, в том числе языка программирования Python и SQL/postgres. Средняя заработная плата для специалистов с опытом работы от 3 до 6 лет в сфере DS составляет около 240 000. Крупнейшее количество вакансий и работодателей находится в России, особенно в Москве и Санкт-Петербурге. Одна из крупнейших компаний-работодателей является Яндекс, которая имеет наибольшее количество вакансий и охватывает большое количество регионов. Однако, многие работодатели не указывают сферу деятельности, а также не указывают диапазон заработной платы в описании вакансии, что может затруднить подбор вакансий для IT-специалистов.

# 7. Дополнительные исследования данных

1. Запрос, позволяющий вычислить, если ли зависимость размера средней заработной платы от количества навыков, указанных в вакансии по специалистам DS.

In [220]:
query_7_1 = f'''select
           DISTINCT(length(key_skills) - length(REPLACE(key_skills, CHR(9), '' )) + 1)  "Количество_навыков",
           ROUND(AVG(salary_from), 0) "Зарплата_от",
           ROUND(AVG(salary_to), 0) "Зарплата_до",
           ROUND(AVG(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0) "Средняя_ЗП"        
           from public.vacancies
           WHERE (name ILIKE '%data scientist%' 
                or (name LIKE '%ML%' and name NOT ILIKE '%HTML%')
                or name ILIKE '%исследователь данных%'
                or name ILIKE '%machine learning%' 
                or name ILIKE '%машинн%обучен%' 
                or name ILIKE '%data science%')
                and key_skills IS NOT NULL
                and (salary_from IS NOT NULL and salary_to IS NOT NULL)
          GROUP BY length(key_skills) - length(REPLACE(key_skills, CHR(9), '' )) + 1
          ORDER BY 4 DESC
           ''' 

In [221]:
df_7_1 = pd.read_sql_query(query_7_1, connection)
df_7_1

,Количество_навыков,Зарплата_от,Зарплата_до,Средняя_ЗП
0,4,200000.0,700000.0,450000.0
1,2,200000.0,250000.0,225000.0
2,3,160000.0,272000.0,216000.0
3,16,170000.0,240000.0,205000.0
4,5,160000.0,241667.0,200833.0
5,6,126000.0,236000.0,181000.0
6,8,125000.0,180000.0,152500.0
7,7,115000.0,172000.0,143500.0
8,11,97638.0,146456.0,122047.0
9,9,61024.0,73228.0,67126.0


Выводы:

Согласно полученным данным, наибольшая средняя зарплата в размере 450 000 указана в вакансиях с 4 ключевыми навыками.

На втором и третем месте по размеру заработной платы находятся вакансии, в которых указано 2 и 3 ключевых навыка соответственно, при этом средняя зарплата составляет около 220 000. Разница между 2 местом и 1 составлят 100%.

На 4 месте расположились вакансии с самым большим набором ключевых навыков - 16 навыков. При этом разница в размере заработной платы между 4 и 3 местом составляет около 5%.

На последнм месте находятся вакансии с самой маленькой средней зарплатой в размере около 70 000, при этом количество ключевых навыков 9.

___

2. Запрос, позволяющий определить, если ли зависимость средней заработной платы специалистов DS в зависимости от сферы деятельности работодателя

In [222]:
query_7_2 = f'''select
           DISTINCT(i.name)  "Сферы_деятельности",
           ROUND(AVG(v.salary_from), 0) "Зарплата_от",
           ROUND(AVG(v.salary_to), 0) "Зарплата_до",
           ROUND(AVG(coalesce((v.salary_from+v.salary_to)/2, v.salary_from, v.salary_to)), 0) "Средняя_ЗП"        
           from public.vacancies v
           join public.employers_industries e on e.employer_id = v.employer_id
           join public.industries i on i.id = e.industry_id 
           WHERE (v.name ILIKE '%data scientist%' 
                or (v.name LIKE '%ML%' and v.name NOT ILIKE '%HTML%')
                or v.name ILIKE '%исследователь данных%'
                or v.name ILIKE '%machine learning%' 
                or v.name ILIKE '%машинн%обучен%' 
                or v.name ILIKE '%data science%')
                and (v.salary_from IS NOT NULL and v.salary_to IS NOT NULL)
          GROUP BY 1
          ORDER BY 4 DESC
           ''' 

In [223]:
df_7_2 = pd.read_sql_query(query_7_2, connection)
df_7_2

,Сферы_деятельности,Зарплата_от,Зарплата_до,Средняя_ЗП
0,"Маркетинговые, рекламные, BTL, дизайнерские, E...",300000.0,400000.0,350000.0
1,Консалтинговые услуги,250000.0,300000.0,275000.0
2,Издательская деятельность,198819.0,273228.0,236024.0
3,Разработка программного обеспечения,166000.0,304000.0,235000.0
4,"Системная интеграция, автоматизации технологи...",157102.0,299323.0,228213.0
5,Кадровые агентства,150000.0,300000.0,225000.0
6,"Интернет-компания (поисковики, платежные систе...",174410.0,241614.0,208012.0
7,"Производство мультимедиа, контента, редакторск...",97638.0,146456.0,122047.0
8,"Производство печатной, полиграфической продукции",97638.0,146456.0,122047.0
9,"Вуз, ссуз колледж, ПТУ",80000.0,150000.0,115000.0


Выводы:

Средняя заработная плата специалистов DS напрямую зависимостзависит от сферы деятельности работодателя. Например в такой сфере деятельности, как "Маркетиновые, рекламные, BTL, дизайнерские услуги" средняя заработная плата 350 000.

У работодателя предоставляющие консалтинговые услуги средняя заработная плата DS специалиста составляет 275 000.

Средняя заработная плата DS специалиста в компании, которая занимается издательской деятельностью, составляет около 240 000.

Самая низкая заработная плата в размере 60 000 в вакансии DS специалиста у работодателя, который занимается электронно-вычислительной, оптической, контрольно-измерительной техникой. 

In [224]:
connection.close()